In [ ]:
import numpy as np
import scipy.stats as ss
import elfi
import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
import pyabc
from pyabc.examples import clinic
from pyabc.plots import plot_marginals, plot_particles
%matplotlib notebook

# Clinic Example
![](img/clinic.png)

In [ ]:
simulator = clinic.simulator

In [ ]:
# Set the generating parameters that we will try to infer
lambda0 = 1/10

# Generate some dat9 (using a fixed seed here)
np.random.seed(1228)
y0 = simulator(lambda0)
print(y0)

In [ ]:
closing_time, nr_patients, nr_waiting_patients = clinic.summaries

In [ ]:
# prior over rate parameter
rate = pyabc.Prior('uniform', 0.01,1.5, name='rate')
rate.name

# Our PYABC

In [ ]:
rej = pyabc.RejectionSampler(priors=[rate], simulator=simulator, 
                             summaries=[closing_time, nr_patients, nr_waiting_patients],
                             observation=y0)
smc = pyabc.SMCSampler(priors=[rate], simulator=simulator, 
                             summaries=[closing_time, nr_patients, nr_waiting_patients],
                             observation=y0)
mcmc = pyabc.MCMCSampler(priors=[rate], simulator=simulator, 
                             summaries=[closing_time, nr_patients, nr_waiting_patients],
                             observation=y0)
abcde = pyabc.ABCDESampler(priors=[rate], simulator=simulator, 
                             summaries=[closing_time, nr_patients, nr_waiting_patients],
                             observation=y0)
bolfi = pyabc.BOLFI(priors=[rate], simulator=simulator, 
                             summaries=[closing_time, nr_patients, nr_waiting_patients],
                             observation=y0,domain= [(0.01, 1.5)])

In [ ]:
rej.sample(nr_samples=1000, threshold=10)

In [ ]:
smc.sample(nr_samples=1000, thresholds=[30, 20, 10])

In [ ]:
mcmc.sample(nr_samples=1000, threshold=10, step_size = [0.01])

In [ ]:
abcde.sample(nr_samples=100, nr_groups = 10, nr_iter = 200, burn_in = 100, alpha = 0.1, beta = 0.1, kappa = 0.9)

In [ ]:
bolfi.sample(nr_samples=10000, threshold=0.01, n_chains=10)

In [ ]:
bolfi.bolfi.plot_acquisition()

In [ ]:
plot_marginals(rej, kde=True, normed=True)
plot_marginals(smc, kde=True, normed=True)
plot_marginals(mcmc, kde=True, normed=True)
plot_marginals(abcde, kde=True, normed=True)
plot_marginals(bolfi, kde=True, normed=True)

In [ ]:
plot_marginals(bolfi, kde=True, normed=True)

In [ ]:
# compute posterior by hand and add it to the bolfi plot above
# in order to show that the problem isn't in the MCMC part of BOLFI

import scipy.stats as ss
def loglikelihood(theta, h=0.01):
    # eqn 47 from BOLFI paper
    m, s = bolfi._bolfi.model.predict(theta)
    # F = gaussian cdf, see eqn 28 in BOLFI paper
    return ss.norm.logcdf((h - m) / s).flatten()

logposterior = lambda theta: loglikelihood(np.atleast_1d(theta)) + rate.logpdf(theta)

x = np.linspace(0,1.5)
y = [np.exp(logposterior(x_i))[0] for x_i in x]
plt.plot(x,y)
plt.show()

In [ ]:
plot_marginals(smc, plot_all=True, normed=True, kde=True)

In [ ]:
plot_particles(smc, equal_axes=False, alpha=0.1)

In [ ]:
plot_particles(smc, as_circles=False, equal_axes=False, alpha=0.1)

In [ ]:
plt.figure()
plt.plot(smc.Thetas, smc.weights[-1], "o")